In [8]:
from bs4 import BeautifulSoup
import requests
import datetime
import pandas as pd

def scrape_data_from_url(URL):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "DNT": "1",
        "Connection": "close",
        "Upgrade-Insecure-Requests": "1"
    }

    page = requests.get(URL, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Use a set to store unique sets based on both date and numbers
    unique_sets_with_dates = set()

    date_elements = soup.find_all('h2')

    for date_element in date_elements:
        date_text = date_element.get_text(strip=True)
        date = datetime.datetime.strptime(date_text, '%a, %b %d, %Y').strftime('%Y-%m-%d')  # Convert to a common date format

        # Find the associated numbers for each date
        number_set_with_date = date_element.find_next(class_='game-balls')
        numbers = tuple([li.get_text(strip=True) for li in number_set_with_date.find_all('li')])

        # Add the tuple (date, numbers) to the set
        unique_sets_with_dates.add((date, numbers))

    # Create a DataFrame from the unique sets
    df = pd.DataFrame(list(unique_sets_with_dates), columns=['Date', 'Game_balls'])

    # Sort the DataFrame in descending order based on the "Date" column
    df = df.sort_values(by='Date', ascending=False)

    # Reset the index sequentially
    df = df.reset_index(drop=True)

    # Split the 'Game_balls' column into separate columns
    df[['Number_1', 'Number_2', 'Number_3', 'Number_4', 'Number_5', 'Number_6', 'Number_7', 'Number_8', 'Number_9', 'Number_10', 'Number_11', 'Number_12', 'Number_13', 'Number_14', 'Number_15', 'Number_16', 'Number_17', 'Number_18', 'Number_19', 'Number_20']] = pd.DataFrame(df['Game_balls'].tolist(), index=df.index)

    # Drop the original 'Game_balls' column
    df = df.drop(columns=['Game_balls'])

    return df

# Example usage:
URL1 = 'https://www.walottery.com/winningnumbers/PastDrawings.aspx?gamename=dailykeno&unittype=year&unitcount=2022'
URL2 = 'https://www.walottery.com/winningnumbers/PastDrawings.aspx?gamename=dailykeno&unittype=year&unitcount=2023'
URL3 = 'https://www.walottery.com/winningnumbers/PastDrawings.aspx?gamename=dailykeno&unittype=year&unitcount=2024'

df1 = scrape_data_from_url(URL1)
df2 = scrape_data_from_url(URL2)
df3 = scrape_data_from_url(URL3)

# Combine dataframes if needed
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

#print(combined_df.head())
# Export the combined DataFrame to an Excel file
excel_filename = 'C:/Users/HP/Documents/Data Analytics Projects/lottery3.xlsx'

combined_df.to_excel(excel_filename, index=False)

print(f"Combined DataFrame exported to {excel_filename}")


Combined DataFrame exported to C:/Users/HP/Documents/Data Analytics Projects/lottery3.xlsx
